In [1]:
# utils.py
import numpy as np
import matplotlib.pylab as plt
from scipy.stats import multivariate_normal


# From https://stackoverflow.com/questions/40642061/how-to-set-axis-ticks-in-multiples-of-pi-python-matplotlib

def multiple_formatter(denominator=2, number=np.pi, latex='\pi'):
    def gcd(a, b):
        while b:
            a, b = b, a%b
        return a
    def _multiple_formatter(x, pos):
        den = denominator
        num = np.int(np.rint(den*x/number))
        com = gcd(num,den)
        (num,den) = (int(num/com),int(den/com))
        if den==1:
            if num==0:
                return r'$0$'
            if num==1:
                return r'$%s$'%latex
            elif num==-1:
                return r'$-%s$'%latex
            else:
                return r'$%s%s$'%(num,latex)
        else:
            if num==1:
                return r'$\frac{%s}{%s}$'%(latex,den)
            elif num==-1:
                return r'$\frac{-%s}{%s}$'%(latex,den)
            else:
                return r'$\frac{%s%s}{%s}$'%(num,latex,den)
    return _multiple_formatter


class Multiple:
    def __init__(self, denominator=2, number=np.pi, latex='\pi'):
        self.denominator = denominator
        self.number = number
        self.latex = latex

    def locator(self):
        return plt.MultipleLocator(self.number / self.denominator)

    def formatter(self):
        return plt.FuncFormatter(multiple_formatter(self.denominator, self.number, self.latex))

In [2]:
#linear_kernel_study.py
import numpy as np


j = 1e-6
for i in range(1, 3):
    jitter = np.eye(i) * j

    X = np.array(list(range(i, 0, -1)))
    X = np.expand_dims(X, axis=1)
    B = np.matmul(X, X.T)

    B = B + jitter

    B_inv = np.linalg.inv(B)

    C = (i+1) * X.T


    C_B_inv = np.matmul(C, B_inv)

    A = np.array([(i+1) * (i+1)])

    sigma = A - np.matmul(C_B_inv, C.T)
    print(i, C_B_inv, sigma)


1 [[1.999998]] [[3.999996e-06]]
2 [[1.19999976 0.59999988]] [[1.80036295e-06]]
